In [3]:
# Load data from MySQL to perform exploratory data analysis
import settings
import mysql.connector
import pandas as pd
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="password",
    database="TwitterDB",
    charset = 'utf8'
)
df = pd.read_sql('SELECT * FROM {}'.format(settings.TABLE_NAME), con=db_connection)

In [4]:
df.head()

,id_str,created_at,text,polarity,subjectivity,user_created_at,user_location,user_description,user_followers_count,longitude,latitude,retweet_count,favorite_count
0,1158824118327939074,2019-08-06 19:36:14,RT _Lib: When thinking about what can be done ...,0,0,2011-11-28 19:34:06,"Charleston, SC",Promoting Information Security Awareness and P...,131,None,None,0,0
1,1158825065233956864,2019-08-06 19:40:00,Data Visualisation with Tableau (article) - Da...,0,0,2013-09-05 02:24:03,"New York, NY","Director, Data Science & Analytics @JetBlue | ...",580,None,None,0,0
2,1158825397754183681,2019-08-06 19:41:19,RT @KirkDBorne: Comparison of top #DataScience...,1,1,2014-01-09 09:30:45,Algiers,"#PhD researcher, #Data believer, #Rstats, Powe...",67,None,None,0,0
3,1158825414644645889,2019-08-06 19:41:23,RT @feeonline: Science writer Neil deGrasse Ty...,0,0,2009-08-10 23:32:26,"Orlando, FL",Founder of Backbone Technologies (http://www.g...,125,None,None,0,0
4,1158825416200769542,2019-08-06 19:41:23,RT @simplivllc: Best Courses in Information Te...,1,1,2018-11-30 11:19:59,None,PICOINGINEERING N.A.S.A. URANUS,493,None,None,0,0


In [ ]:
import seaborn as sns
sns.set(style="whitegrid")
ax = sns.violinplot(x=df["polarity"])